In [656]:
import pandas as pd
import numpy as np

In [657]:
routes_df = pd.read_csv("../data/routes.csv")
countries_df = pd.read_csv("../data/countries.csv")
airlines_df = pd.read_csv("../data/airlines.csv")

## Clean routes

In [658]:
# Drop unnecessary columns
try:
    routes_df = routes_df.drop(columns=["no_of_stops"])
except KeyError:
    pass

# Drop rows with null values (\N in data)
routes_df = routes_df[routes_df["destination_airport_id"] != '\\N']
routes_df = routes_df[routes_df["source_airport_id"] != '\\N']
routes_df = routes_df[routes_df["airline_id"] != '\\N']

# Type cast objects to string
routes_df["airline"] = pd.Series(routes_df["airline"], dtype="string")
routes_df["airline_id"] = pd.Series(routes_df["airline_id"], dtype="int64")
routes_df["source_airport"] = pd.Series(routes_df["source_airport"], dtype="string")
routes_df["source_airport_id"] = pd.Series(routes_df["source_airport_id"], dtype="int64")
routes_df["destination_airport"] = pd.Series(routes_df["destination_airport"], dtype="string")
routes_df["destination_airport_id"] = pd.Series(routes_df["destination_airport_id"], dtype="int64")

## Clean countries

In [659]:
# Remove NaN rows
countries_df = countries_df[pd.notna(countries_df["dafif_code"])]

# Drop rows with null values (\N in data) 
countries_df[countries_df["iso_code"] != "\\N"]

# Type cast objects to string
countries_df["name"] = pd.Series(countries_df["name"], dtype="string")
countries_df["iso_code"] = pd.Series(countries_df["iso_code"], dtype="string")
countries_df["dafif_code"] = pd.Series(countries_df["dafif_code"], dtype="string")

## Clean airlines

In [660]:
# Drop rows with null values (\N in data)
try:
    airlines_df = airlines_df.drop(columns=["alias"])
except KeyError:
    pass

airlines_df = airlines_df[airlines_df["icao_code"] != "\\N"]

# Remove NaN rows
airlines_df = airlines_df[pd.notna(airlines_df["call_sign"])]
airlines_df = airlines_df[pd.notna(airlines_df["iata_code"])]

# Type cast objects to string
airlines_df["id"] = pd.Series(airlines_df["id"], dtype="int64")
airlines_df["name"] = pd.Series(airlines_df["name"], dtype="string")
airlines_df["iata_code"] = pd.Series(airlines_df["iata_code"], dtype="string")
airlines_df["icao_code"] = pd.Series(airlines_df["icao_code"], dtype="string")
airlines_df["call_sign"] = pd.Series(airlines_df["call_sign"], dtype="string")
airlines_df["country"] = pd.Series(airlines_df["country"], dtype="string")
airlines_df["active"] = pd.Series(airlines_df["active"], dtype="string")

## Join airlines and countries

In [661]:
airline_plus = pd.merge(airlines_df, countries_df, left_on="country", right_on="name")

In [662]:
try:
    airline_plus["country"] = airline_plus["name_y"]
    airline_plus["airline"] = airline_plus["name_x"]
    airline_plus = airline_plus.drop(columns=["name_x", "name_y"])
except KeyError:
    pass

## Create airlines per country CSV by doing groupby

In [663]:
# Which country has highest number of airlines ?
num_of_airlines_gby = airline_plus.groupby("country")["airline"].count()
num_of_airlines_df = pd.DataFrame(num_of_airlines_gby.items(), columns=["country", "num_of_airlines"])
num_of_airlines_df.to_csv("../data/airlines_per_country.csv", index=False)

## Join airlines and routes

In [664]:
# Which airline has highest number of routes ?
routes_plus = pd.merge(airlines_df, routes_df, left_on="id", right_on="airline_id")

## Create routes per airline CSV by doing groupby

In [665]:
num_of_routes_gby = routes_plus.groupby("name")["airline_id"].count()
num_of_routes_df = pd.DataFrame(num_of_routes_gby.items(), columns=["airline", "num_of_routes"])
num_of_routes_df.to_csv("../data/routes_per_airline.csv", index=False)

In [666]:
airline_plus.groupby("country")["airline"].count()

country
Afghanistan            3
Albania                2
Algeria                3
Angola                 1
Antigua and Barbuda    2
                      ..
Venezuela              7
Vietnam                3
Yemen                  1
Zambia                 1
Zimbabwe               2
Name: airline, Length: 166, dtype: int64

In [667]:
airline_plus

,id,iata_code,icao_code,call_sign,country,active,iso_code,dafif_code,airline
0,3,1T,RNX,NEXTIME,South Africa,Y,ZA,SF,1Time Airline
1,1829,MN,CAW,COMMERCIAL,South Africa,Y,ZA,SF,Comair
2,2881,D6,ILN,INLINE,South Africa,Y,ZA,SF,Interair South Africa
3,2896,ID,ITK,INTERLINK,South Africa,Y,ZA,SF,Interlink Airlines
4,3393,JE,MNO,TULCA,South Africa,Y,ZA,SF,Mango
...,...,...,...,...,...,...,...,...,...
1060,16624,TW,TWB,TWAY AIR,South Korea,Y,KR,KS,Tway Airlines
1061,16707,HN,HNX,HNX,South Korea,Y,KR,KS,Hankook Airline
1062,11840,8F,STP,SAOTOME AIRWAYS,Sao Tome and Principe,Y,ST,TP,STP Airways
1063,17780,DF,MJG,MJG,Puerto Rico,Y,PR,RQ,Michael Airlines


In [668]:
routes_plus

,id,name,iata_code,icao_code,call_sign,country,active,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,equipment
0,10,40-Mile Air,Q5,MLA,MILE-AIR,United States,Y,Q5,10,FAI,3832,HKB,7242,CNA
1,10,40-Mile Air,Q5,MLA,MILE-AIR,United States,Y,Q5,10,HKB,7242,FAI,3832,CNA
2,21,Aigle Azur,ZI,AAF,AIGLE AZUR,France,Y,ZI,21,AAE,220,MRS,1353,319
3,21,Aigle Azur,ZI,AAF,AIGLE AZUR,France,Y,ZI,21,AAE,220,ORY,1386,319
4,21,Aigle Azur,ZI,AAF,AIGLE AZUR,France,Y,ZI,21,ALG,210,CDG,1382,319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62599,20160,City Airways,E8,GTA,CITY AIR,Thailand,Y,E8,20160,DMK,3157,CSX,3371,734
62600,20160,City Airways,E8,GTA,CITY AIR,Thailand,Y,E8,20160,DMK,3157,HKT,3179,734
62601,20160,City Airways,E8,GTA,CITY AIR,Thailand,Y,E8,20160,DMK,3157,KHN,3384,734
62602,20160,City Airways,E8,GTA,CITY AIR,Thailand,Y,E8,20160,HKT,3179,DMK,3157,734
